In [2]:
!activate ds


CLINK (ds) CLINK CLINK CLINK C:\Users\Christian\Desktop\Nasa\Model_1_short_segmentation\Training_code\code\src\sample\submission>conda.bat activate ds 


In [1]:
import sys

import numpy as np 
import sys,os
import cv2
import matplotlib.pyplot as plt
import argparse
import pandas as pd
from utils import *
import random
from operator import itemgetter

In [2]:
root_dir = "/home/sebasmos/Documentos/NASA_Spacesuit/test/"

In [4]:
root_dir

'/home/sebasmos/Documentos/NASA_Spacesuit/test/'

In [3]:
# Load data paths
training_images = []   # images png + jpg
img_ann = []        # images annotations
    
videos = []         # videos
videos_ann = []     # videos annotations
video_frames = []   # video frames
video_frames_ann=[] # video frames annotations

for root, dirs, files in os.walk(root_dir, topdown=True):
    for name in files:
        path = os.path.join(root, name)            
        if '.png' in path  and 'images' in path:
            training_images.append(path)
            print(path)
        if '.jpg' in path  and 'images' in path:
            training_images.append(path)
        if '.txt' in path and 'images' in path:
            img_ann.append(path)
        if '.mov' in path and 'videos' in path:
            videos.append(path)
        if '.csv' in path and 'videos' in path:
            videos_ann.append(path)
        if '.jpg' in path and 'videos-frames' in path:
            video_frames.append(path)
            #if '.csv' in path and 'videos-frames' in path: #TODO: CHECK ANNOTATIONS
            #    video_frames_ann.append(path)
print('Image training instances: ', len(training_images))
print('videos training instances: ', len(videos))
print('video_frames evaluation instances: ', len(video_frames))


/home/sebasmos/Documentos/NASA_Spacesuit/test/images/003640.png
/home/sebasmos/Documentos/NASA_Spacesuit/test/images/731747.png
/home/sebasmos/Documentos/NASA_Spacesuit/test/images/473622.png
/home/sebasmos/Documentos/NASA_Spacesuit/test/images/303997.png
/home/sebasmos/Documentos/NASA_Spacesuit/test/images/794528.png
/home/sebasmos/Documentos/NASA_Spacesuit/test/images/083823.png
/home/sebasmos/Documentos/NASA_Spacesuit/test/images/682902.png
/home/sebasmos/Documentos/NASA_Spacesuit/test/images/403277.png
/home/sebasmos/Documentos/NASA_Spacesuit/test/images/354447.png
/home/sebasmos/Documentos/NASA_Spacesuit/test/images/739196.png
/home/sebasmos/Documentos/NASA_Spacesuit/test/images/039383.png
/home/sebasmos/Documentos/NASA_Spacesuit/test/images/617674.png
/home/sebasmos/Documentos/NASA_Spacesuit/test/images/939658.png
/home/sebasmos/Documentos/NASA_Spacesuit/test/images/080202.png
/home/sebasmos/Documentos/NASA_Spacesuit/test/images/457481.png
/home/sebasmos/Documentos/NASA_Spacesuit

In [54]:
video_frames

['/home/sebasmos/Documentos/NASA_Spacesuit/train/videos-frames/EMU-12-0282.jpg',
 '/home/sebasmos/Documentos/NASA_Spacesuit/train/videos-frames/MKIII-06-0502.jpg',
 '/home/sebasmos/Documentos/NASA_Spacesuit/train/videos-frames/MKIII-06-0168.jpg',
 '/home/sebasmos/Documentos/NASA_Spacesuit/train/videos-frames/MKIII-24-0130.jpg',
 '/home/sebasmos/Documentos/NASA_Spacesuit/train/videos-frames/EMU-05-0170.jpg',
 '/home/sebasmos/Documentos/NASA_Spacesuit/train/videos-frames/MKIII-04-0201.jpg',
 '/home/sebasmos/Documentos/NASA_Spacesuit/train/videos-frames/MKIII-16-0344.jpg',
 '/home/sebasmos/Documentos/NASA_Spacesuit/train/videos-frames/EMU-09-0196.jpg',
 '/home/sebasmos/Documentos/NASA_Spacesuit/train/videos-frames/EMU-04-0539.jpg',
 '/home/sebasmos/Documentos/NASA_Spacesuit/train/videos-frames/EMU-12-0978.jpg',
 '/home/sebasmos/Documentos/NASA_Spacesuit/train/videos-frames/EMU-09-0094.jpg',
 '/home/sebasmos/Documentos/NASA_Spacesuit/train/videos-frames/MKIII-06-0385.jpg',
 '/home/sebasmos

# Image segmentation  

In [6]:

def segmentation(path, label):
    
    flag = False 

    IMG_SIZE = 500
    
    image = cv2.imread(path)

    mask = np.zeros(image.shape[:2], np.uint8) # create a similar mask image

    backgroundModel = np.zeros((1, 65), np.float64) # These are arrays used by the algorithm internally. You just create two np.float64 type zero arrays of size (1,65).
    
    foregroundModel = np.zeros((1, 65), np.float64) # These are arrays used by the algorithm internally. You just create two np.float64 type zero arrays of size (1,65).

    rectangle = (50,50,450,290)

    cv2.grabCut(image, mask, rectangle,  
                backgroundModel, foregroundModel,
                3, cv2.GC_INIT_WITH_RECT)

    mask2 = np.where((mask==2)|(mask==0),0,1).astype('uint8') # apply mask
    # So we modify the mask such that all 0-pixels and 2-pixels are put to 0 (ie background) and all 1-pixels and 3-pixels are put to 1(ie foreground pixels). Now our final mask is ready. Just multiply it with input image to get the segmented image. 
    #print(mask2)
    img = image*mask2[:,:,np.newaxis]
    
    if (np.mean(mask2)!= 0):
        img = image*mask2[:,:,np.newaxis]

        [rows,cols]= mask2.shape

        triplets = []
        for i in range(rows):
            for j in range(cols):
                k = mask2[i,j]
                if k == 1:
                    aux = [i,j,int(random.uniform(0,3))]

                    triplets.append(aux)
                    #print(triplets)
                    #mask[i,j] = 1
                else: # ADD HERE LABEL 1 AND LABEL 0
                    aux = 0
                    #print("no")
                    #mask[i,j] = 0

        #-----------------Select 15 TRIPLETS FROM SEGMENTED IMAGE----------------------------
        points = []
        # CALCULATE THE 15 TRIPLETS COORDINATES THAT HAVE THE MAX DIST WITH dist = numpy.linalg.norm(a-b)
        # Body-part detection 
        for i in range(15):
            points.append(int(random.triangular(0,len(triplets)))) # len(triplets) bc of all possible triplets on segmentated img

        #-----------------15 TRIPLETS SELECTION-------------------------------------
        sel  = itemgetter(*points)(triplets)
        #print("Selected 15 triplets: ", len(sel))
        #print(sel)

        #----------------SELECT AREAS---------------------------------------------------------
        #print("x ", min(sel))
        triplet_min = min(sel)
        #print("y ", max(sel))
        triplet_max = max(sel)
        x_i,y_i,l_i = triplet_min
        x_f,y_f,l_f = triplet_max

        c1 = x_i,y_i
        c2 = x_f,y_i
        c3 = x_f,y_f
        c4 = x_i,y_f

        rect = c1 + c2 + c3 + c4 + c1 # 5 pairs 
        #print("Polygon coordenates ",rect)

        #plt.imshow(img)
        #plt.colorbar()
        #plt.show()
        #print(final)
        flag = False # Keep false of there is detection
        #print('{} - {}'.format(sel,rect))
    else:
        print("no obj detected")
        sel = []
        rect = []
        flag = True # turn to truth if there is not detection
        #print('{} - {}'.format(sel,rect))
    
    #plt.imshow(mask2)
    
    return sel, rect, flag
# Fast testing bef assignmt



In [7]:
def feature_extraction(path, label): # ADAPT ANNOTATIONS CORRECTLY & POLYGON + AREA EXTRACTION
  #ima = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
  
  triplets, coordinates,flag = segmentation(path, label)
  
  triplets = np.array( [triplets]  ).ravel().ravel()  #+ coordinates
  triplets = list(triplets)
  return triplets, coordinates,flag

# # Fast testing bef assignmt
# path = "/home/sebasmos/Documentos/NASA_Spacesuit/test/images/539372.png"

# triplets, coordinates,flag = feature_extraction(path, 1)
# coordinates

In [8]:
i = 0
# Training with images 
Num_features = 1
feat_names = ['Polygon'] # TODO: EXTRACT TRIPLETS HERE!
train_size = len(training_images)
final = []

data_train = []# np.zeros((train_size, Num_features+1), dtype=np.float32)
#data_test = np.zeros((test_size, Num_features+1), dtype=np.float32)

#--------------------------IMAGES-----------------------------------------------------------
print("Creating solutions/images/annotations/solutions.csv...")
for path in training_images:
    print('    {}/{}...'.format(i+1, len(training_images)))
    
    triplets, coordinates,flag = feature_extraction(path, 2)
    
    id_img = path[len(path) - 10:len(path)]
    
    if flag==True:
        row = id_img
    else:
        row = id_img + ',' +str(triplets)[1:-1] + ','+ '[' + str(coordinates) + ']' # working with lists
    print(' {} '.format(row))
    
    #data_train.append(feature_extraction(path, 2))
    # Extract id_name from image

    # Add id + triplets + geoms
    row_str = " ".join(map(str, row))
    file1 = open("solutions_final.csv","a")
    file1.write(row + " \n")
    file1.close()
    i = i+1

Creating solutions/images/annotations/solutions.csv...
    1/84...
no obj detected
 003640.png 
    2/84...
 049790.jpg,95, 403, 2, 100, 101, 2, 112, 317, 2, 109, 233, 2, 140, 74, 0, 148, 197, 1, 70, 193, 1, 111, 141, 1, 126, 165, 2, 106, 291, 1, 91, 220, 1, 132, 171, 0, 85, 285, 2, 116, 190, 2, 90, 133, 0,[(70, 193, 148, 193, 148, 197, 70, 197, 70, 193)] 
    3/84...
 731747.png,250, 327, 0, 178, 183, 1, 247, 272, 0, 228, 196, 1, 214, 226, 1, 160, 185, 0, 131, 228, 2, 210, 188, 1, 223, 175, 2, 272, 179, 0, 182, 229, 2, 250, 359, 0, 224, 76, 2, 225, 127, 0, 201, 106, 2,[(131, 228, 272, 228, 272, 179, 131, 179, 131, 228)] 
    4/84...
no obj detected
 473622.png 
    5/84...
 303997.png,131, 303, 1, 312, 302, 0, 228, 262, 0, 264, 99, 0, 207, 166, 1, 177, 230, 2, 200, 88, 1, 124, 245, 2, 246, 183, 1, 249, 141, 0, 217, 212, 1, 258, 304, 1, 292, 103, 1, 262, 328, 2, 226, 234, 0,[(124, 245, 312, 245, 312, 302, 124, 302, 124, 245)] 
    6/84...


KeyboardInterrupt: 

# Video segmentation  - different format

In [12]:

def video_segmentation(path, label):
    
    flag = False 

    IMG_SIZE = 500
    
    image = cv2.imread(path)

    mask = np.zeros(image.shape[:2], np.uint8) # create a similar mask image

    backgroundModel = np.zeros((1, 65), np.float64) # These are arrays used by the algorithm internally. You just create two np.float64 type zero arrays of size (1,65).
    
    foregroundModel = np.zeros((1, 65), np.float64) # These are arrays used by the algorithm internally. You just create two np.float64 type zero arrays of size (1,65).

    rectangle = (50,50,450,290)

    cv2.grabCut(image, mask, rectangle,  
                backgroundModel, foregroundModel,
                3, cv2.GC_INIT_WITH_RECT)

    mask2 = np.where((mask==2)|(mask==0),0,1).astype('uint8') # apply mask
    # So we modify the mask such that all 0-pixels and 2-pixels are put to 0 (ie background) and all 1-pixels and 3-pixels are put to 1(ie foreground pixels). Now our final mask is ready. Just multiply it with input image to get the segmented image. 
    #print(mask2)
    img = image*mask2[:,:,np.newaxis]
    
    if (np.mean(mask2)!= 0):
        img = image*mask2[:,:,np.newaxis]

        [rows,cols]= mask2.shape

        triplets = []
        for i in range(rows):
            for j in range(cols):
                k = mask2[i,j]
                if k == 1:
                    aux = [i,j,int(random.uniform(0,cols))]

                    triplets.append(aux)
                    #print(triplets)
                    #mask[i,j] = 1
                else: # ADD HERE LABEL 1 AND LABEL 0
                    aux = 0
                    #print("no")
                    #mask[i,j] = 0

        #-----------------Select 15 TRIPLETS FROM SEGMENTED IMAGE----------------------------
        points = []
        # CALCULATE THE 15 TRIPLETS COORDINATES THAT HAVE THE MAX DIST WITH dist = numpy.linalg.norm(a-b)
        # Body-part detection 
        for i in range(13):
            points.append(int(random.triangular(0,len(triplets)))) # len(triplets) bc of all possible triplets on segmentated img

        #-----------------15 TRIPLETS SELECTION-------------------------------------
        sel  = itemgetter(*points)(triplets)
        #print("Selected 15 triplets: ", len(sel))
        #print(sel)

        #----------------SELECT AREAS---------------------------------------------------------
        #print("x ", min(sel))
        triplet_min = min(sel)
        #print("y ", max(sel))
        triplet_max = max(sel)
        x_i,y_i,l_i = triplet_min
        x_f,y_f,l_f = triplet_max

        c1 = x_i,y_i
        c2 = x_f,y_i
        c3 = x_f,y_f
        c4 = x_i,y_f

        rect = c1 + c2 + c3 + c4 + c1 # 5 pairs 
        #print("Polygon coordenates ",rect)

        #plt.imshow(img)
        #plt.colorbar()
        #plt.show()
        #print(final)
        flag = False # Keep false of there is detection
        #print('{} - {}'.format(sel,rect))
    else:
        print("no obj detected")
        sel = []
        rect = []
        flag = True # turn to truth if there is not detection
        #print('{} - {}'.format(sel,rect))
    
    #plt.imshow(mask2)
    
    
    return sel, flag

def video_feature_extraction(path, label): # ADAPT ANNOTATIONS CORRECTLY & POLYGON + AREA EXTRACTION
  #ima = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
  triplets, flag = video_segmentation(path, label)
  
  triplets = np.array( [triplets]  ).ravel().ravel()  #+ coordinates
  triplets = list(triplets)
  return triplets, flag

# Fast testing bef assignmt
# path = "/home/sebasmos/Documentos/NASA_Spacesuit/train/videos-frames/MKIII-06-0111.jpg"
# triplets, coordinates,flag = video_feature_extraction(path, 1)

# EXTRACT anotations solutions

In [8]:

     videos = extractVideoNames(video_frames)
     videos_indexes_dict = {vid:1 for vid in videos}
     print(videos)
     topheader = 'Frame #,CLAV,,,RSJC,,,LSJC,,,REJC,,,LEJC,,,RWJC,,,LWJC,,,RHJC,,,LHJC,,,RKJC,,,LKJC,,,RAJC,,,LAJC,,'
     header = ['']+['X','Y','Z']*13
     path = ''
     for frame_path in video_frames:
        
         for v in videos:
             if v in frame_path:
                 anotations_path = "{}".format(root_dir)
                 # create anotations folder if not exists
                 try:
                     os.makedirs(anotations_path)
                 except: 
                     pass
                 path_ = "{}{}{}".format(anotations_path,os.sep,v+".csv")            
                 
                 path = frame_path
                 header_str = ",".join(header)+"\n"
                 file1 = open(path_,"a")
                 if videos_indexes_dict[v]==1:
                     print(topheader + " \n" + header_str)
                     file1.write(topheader + " \n" + header_str)
                 triplets, flag = video_feature_extraction(path , 2)
                 id_img = frame_path[len(path) - 15:len(path)]
    #              id_img=''
    #              if flag==True:
    #                  row = id_img
    #              else:
    #                  row = id_img + ',' +str(triplets)[1:-1]
                 row = str(triplets)[1:-1]
                 print(' {} '.format(row))
                 #record the first column index
                 row = str(videos_indexes_dict[v])+ "," + row
                 row_str = " ".join(map(str, row))
                 file1.write(row + " \n")
                 file1.close()
                 videos_indexes_dict[v]+=1
     print("Finished annotation videos.. ")

['MKIII-22', 'MKIII-05', 'MKIII-10', 'EMU-03', 'MKIII-23', 'EMU-13', 'MKIII-11', 'MKIII-02']
Frame #,CLAV,,,RSJC,,,LSJC,,,REJC,,,LEJC,,,RWJC,,,LWJC,,,RHJC,,,LHJC,,,RKJC,,,LKJC,,,RAJC,,,LAJC,, 
,X,Y,Z,X,Y,Z,X,Y,Z,X,Y,Z,X,Y,Z,X,Y,Z,X,Y,Z,X,Y,Z,X,Y,Z,X,Y,Z,X,Y,Z,X,Y,Z,X,Y,Z

 269, 309, 543, 170, 204, 830, 205, 232, 30, 193, 341, 860, 206, 356, 684, 160, 430, 674, 180, 209, 237, 235, 486, 756, 228, 493, 594, 208, 241, 782, 199, 394, 848, 164, 102, 426, 216, 347, 859 
Frame #,CLAV,,,RSJC,,,LSJC,,,REJC,,,LEJC,,,RWJC,,,LWJC,,,RHJC,,,LHJC,,,RKJC,,,LKJC,,,RAJC,,,LAJC,, 
,X,Y,Z,X,Y,Z,X,Y,Z,X,Y,Z,X,Y,Z,X,Y,Z,X,Y,Z,X,Y,Z,X,Y,Z,X,Y,Z,X,Y,Z,X,Y,Z,X,Y,Z



KeyboardInterrupt: 

train_size = len(training_images)
print("training data: ", train_size)
train_pd = pd.read_csv("/home/sebasmos/Documentos/NASA_Spacesuit/NASA_SpatiumAESS/Training_code/code/src/sample/submission/training_images.csv")

Num_suits = 1 # TO UPDATE ACCORDING TO IMAGE
Triplets = 15 #
aux = Num_suits*Triplets
#joints = np.zeros((train_size, Num_suits*Triplets))

#train ? 
id_images = []
aux = len(path) - 10
for path in training_images:
    id_images.append(path[aux:len(path)])


def gen_array():
    x_f = []
    y_f = []
    v_f = []

    for i in range(15):
        x_f.append(int (random.uniform(100,999)))
        y_f.append(int (random.uniform(100,999)))
        v_f.append(int (random.uniform(0,3)))

    return x_f,y_f,v_f


def gen_geometries():
    x_f = []
    y_f = []
    type = 0
    if type == 0: #rectangular then 8 var, 4 corners
        for i in range(4):
            x_f.append(int (random.uniform(100,999)))
            y_f.append(int (random.uniform(100,999)))
    elif type==1: # 3 corners
        print("triangular")
    else:
        print("no figure detected")
    return x_f,y_f

def generar_csv (imag,coo,polig):

   f=open('../../../../solution/solution.csv','w')

   for l in range (len(imag)):
      
      f.write(str(imag[l])+','+str(coo[l])+','+str(polig[l])+'\n')

   f.close()

joints_id = []
geom = []

for id, j in enumerate(training_images):

    for i in range(Num_suits):
            #print(" es multiplo de 3")
            x_f,y_f,v_f = gen_array()
            joints_id.append(x_f + y_f + v_f)
            x_geom, y_geom = gen_geometries()
            geom.append(x_geom + y_geom)
            
print("id_images: ",train_size)
print("joints_id: ",len(joints_id))
print("geom: ",len(geom))

generar_csv (id_images,joints_id,geom)



